In [1]:
import os
import dotenv
import json
from openai import OpenAI

# Set-up the client
put you're API key in a .env file with the format:

API_KEY = *****


In [5]:
# Load environment variables from .env file
dotenv.load_dotenv()

# Get the API key and set up the client to connect to Metis AI
metis_api_key = os.getenv("API_KEY")
client = OpenAI(
    api_key=metis_api_key,
    base_url="https://api.metisai.ir/openai/v1"
)

# Step 1: Create a Custom Tool (Python Function)
Here, we define a standard Python function. This function is our "tool" that the AI will learn to use. It takes a number (amount), multiplies it by a fixed exchange rate, and returns the result in a JSON format.

In [6]:
# --- Step 1: Define your custom Python function with a parameter ---
def convert_usd_to_irr(amount: int):
    """
    Converts a given USD amount to IRR by multiplying it by a fixed rate.
    """
    # In a real application, this rate could be fetched from a live API
    fixed_rate = 970000
    total_irr = amount * fixed_rate

    # Return the final calculated amount as a JSON object
    return json.dumps({"total_in_irr": total_irr})

# Step 2: Describe the Tool for the AI Model
The AI model cannot read our Python code directly. We must describe the function to it in a specific format called a schema. This schema tells the model:

- name: The function's name.
- description: What the function does, in natural language.
- parameters: The arguments the function needs, their types (number, string, etc.), and which ones are required.

In [7]:
# --- Step 2: Describe the new function and its required parameters to the model ---
functions = [
    {
        "name": "convert_usd_to_irr",
        "description": "یک مبلغ مشخص از دلار آمریکا را به ریال ایران تبدیل می‌کند",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {
                    "type": "number",
                    "description": "مبلغ به دلار برای تبدیل شدن",
                }
            },
            "required": ["amount"],  # The 'amount' parameter is mandatory / پارامتر 'amount' اجباری است
        },
    }
]

# Step 3: Define the User's Prompt
Now, we create the user's question. The model will analyze this text to determine if it needs to use one of the tools we provided in the functions list. Notice how the prompt "۲۰۰ دلار" contains the amount that our function needs.

In [8]:
# --- Step 3: Start the conversation with the user's prompt ---
messages = [
    {"role": "user", "content": "سلام، لطفا ۲۰۰ دلار رو به ریال تبدیل کن"}
]

# --- Step 4: First API call to see if the model wants to use the function ---
print(">>> Calling the model...")
response = client.chat.completions.create(
    model="gpt-4o",  # You can change this to your desired model
    messages=messages,
    functions=functions,
    function_call='auto'
)

response_message = response.choices[0].message

>>> Calling the model...


# Step 4: Execute the Full Function Calling Flow
This is the main logic. It's a two-step API interaction:

First Call: We send the user's message and the function description to the model. The model doesn't answer the user directly. Instead, it responds with a request to call a specific function (convert_usd_to_irr) and the arguments it extracted from the prompt (amount: 200).

Run the Function: Our Python code receives this request, runs our local convert_usd_to_irr function with the provided arguments, and captures the output.

Second Call: We call the API again. This time, we include the original prompt, the model's first response, and the result from our Python function. With all this context, the model generates a final, human-friendly answer.

In [9]:
# --- Step 5: Check if the model decided to call our function ---
if response_message.function_call:
    function_name = response_message.function_call.name
    print(f">>> Model decided to call the function '{function_name}'.")

    # --- Step 6: Extract arguments and execute the function ---

    # Parse the JSON string of arguments the model provides
    arguments = json.loads(response_message.function_call.arguments)
    print(f">>> With arguments: {arguments}")

    # Call the function with the extracted arguments
    function_output = convert_usd_to_irr(amount=arguments.get("amount"))
    print(f">>> Function output: {function_output}")

    # Add the assistant's response (the function call) to the message history
    messages.append(response_message)

    # --- Step 7: Send the function's result back to the model ---
    messages.append({
        "role": "function",
        "name": function_name,
        "content": function_output
    })

    print(">>> Calling the model again with function result...")
    final_response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    print("\n--- Final Answer ---")
    print(final_response.choices[0].message.content)

else:
    print("\n--- Final Answer ---")
    print(response_message.content)

>>> Model decided to call the function 'convert_usd_to_irr'.
>>> With arguments: {'amount': 200}
>>> Function output: {"total_in_irr": 194000000}
>>> Calling the model again with function result...

--- Final Answer ---
۲۰۰ دلار برابر است با ۱۹۴،۰۰۰،۰۰۰ ریال.
